In [1]:
from pathlib import Path
import numpy as np

In [6]:
root_path = Path("/media/hdd1/i-care")
ae_feat_path = root_path / "ae-features"
agg_ae_feat_path = root_path / "agg-ae-features"
man_feat_path = root_path / "features"
combined_feat_path = root_path / "combined-features"

In [18]:
for patient_dir in ae_feat_path.iterdir():
    (combined_feat_path / patient_dir.name).mkdir(parents=True, exist_ok=True)
    (agg_ae_feat_path / patient_dir.name).mkdir(parents=True, exist_ok=True)

    ae_files_by_hour = {}
    for ae_file in patient_dir.iterdir():
        # Extract hour from the filename
        hour = ae_file.stem.split("_")[1]
        if hour not in ae_files_by_hour:
            ae_files_by_hour[hour] = []
        ae_files_by_hour[hour].append(ae_file)

    for hour, ae_files in ae_files_by_hour.items():
        # Aggregate 5-minute AE features into 1-hour
        ae_features = [np.load(file) for file in sorted(ae_files)]
        aggregated_ae_feat = np.vstack(ae_features)
        agg_ae_filename = f"{patient_dir.name}_{hour}_features.npy"
        np.save(agg_ae_feat_path / patient_dir.name / agg_ae_filename, aggregated_ae_feat[:12])

        # Find corresponding manual feature file
        man_file_pattern = f"{patient_dir.name}_*_{hour}_features.npy"
        man_file = list(man_feat_path.joinpath(patient_dir.name).glob(man_file_pattern))

        if man_file:
            man_file = man_file[0]
            # Load feature array
            man_feat = np.load(man_file)
            # Truncate to the shortest sample
            min_sample = min(aggregated_ae_feat.shape[0], man_feat.shape[0])
            aggregated_ae_feat = aggregated_ae_feat[:min_sample]
            man_feat = man_feat[:min_sample]
            # Concatenate features
            combined_feat = np.hstack((man_feat, aggregated_ae_feat))
            # Save combined features
            combined_filename = f"{patient_dir.name}_{hour}_features.npy"
            np.save(combined_feat_path / patient_dir.name / combined_filename, combined_feat.T)